In [1]:
!pip install biopython
!pip install fair-esm
!pip install h5py
!pip install SeqIO


   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ----------------------- ---------------- 1.6/2.7 MB 9.3 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 10.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
INFO: pip is looking at multiple versions of seqio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of seqio to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might ne

ERROR: Cannot install seqio==0.0.0, seqio==0.0.1, seqio==0.0.10, seqio==0.0.11, seqio==0.0.12, seqio==0.0.13, seqio==0.0.14, seqio==0.0.15, seqio==0.0.16, seqio==0.0.17, seqio==0.0.18, seqio==0.0.19, seqio==0.0.2, seqio==0.0.20, seqio==0.0.3, seqio==0.0.4, seqio==0.0.5, seqio==0.0.6, seqio==0.0.7, seqio==0.0.8 and seqio==0.0.9 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [2]:
import torch
from Bio import SeqIO
import esm
import numpy as np
from pathlib import Path
import gc
import pickle
import time
import os
from tqdm import tqdm
import shutil
import json

original_torch_load = torch.load

def patched_torch_load(*args, **kwargs):
    kwargs.setdefault('weights_only', False)
    return original_torch_load(*args, **kwargs)

torch.load = patched_torch_load

In [4]:
# Specify your model path
model_path = r"C:\Users\chitaxxx\OneDrive - Orion Corporation\Kaggle\cafa-6-protein-function-prediction\Model ESM2\hub\checkpoints\esm2_t33_650M_UR50D.pt"

# Load the model and alphabet from the local checkpoint, setting weights_only=False
model, alphabet = esm.pretrained.load_model_and_alphabet_local(model_path)
torch.load = original_torch_load

# Set up the batch converter
batch_converter = alphabet.get_batch_converter()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()  # Set to evaluation mode

print(f"Model loaded successfully on {device}")

Model loaded successfully on cpu


In [5]:
# Path to your FASTA file
fasta_path = r"C:\Users\chitaxxx\OneDrive - Orion Corporation\Kaggle\cafa-6-protein-function-prediction\Train\train_sequences.fasta"

# Read all sequences
sequences = []
for record in SeqIO.parse(fasta_path, "fasta"):
    sequences.append((record.id, str(record.seq)))

print(f"Total sequences loaded: {len(sequences)}")


Total sequences loaded: 82404


In [ ]:
def extract_embeddings_batch_with_progress(sequence_list, model, batch_converter, device, batch_size=8):
    """
    Extract embeddings for a batch of sequences with progress bar
    
    Args:
        sequence_list: List of tuples (id, sequence)
        model: ESM model
        batch_converter: Batch converter from alphabet
        device: torch device
        batch_size: Number of sequences to process at once
    
    Returns:
        Dictionary with sequence IDs as keys and embeddings as values
    """
    embeddings_dict = {}
    
    total_batches = (len(sequence_list) + batch_size - 1) // batch_size
    
    # Create progress bar
    pbar = tqdm(total=len(sequence_list), desc="Processing sequences", 
                unit="seq", ncols=100)
    
    # Process in batches
    for i in range(0, len(sequence_list), batch_size):
        batch = sequence_list[i:i+batch_size]
        
        # Convert batch
        batch_labels, batch_strs, batch_tokens = batch_converter(batch)
        batch_tokens = batch_tokens.to(device)
        
        # Extract embeddings
        with torch.no_grad():
            results = model(batch_tokens, repr_layers=[33], return_contacts=False)  # Use last layer
            token_embeddings = results["representations"][33]
        
        # Store embeddings (mean pooling over sequence length, excluding special tokens)
        for j, (label, seq) in enumerate(batch):
            # Get embedding for this sequence (exclude BOS and EOS tokens)
            seq_len = len(seq)
            embedding = token_embeddings[j, 1:seq_len+1].mean(0)  # Mean pooling
            embeddings_dict[label] = embedding.cpu().numpy()
        
        # Clear memory
        del batch_tokens, results, token_embeddings
        torch.cuda.empty_cache()
        
        # Update progress bar
        pbar.update(len(batch))
    
    pbar.close()
    
    return embeddings_dict

In [9]:
from collections import defaultdict

def group_sequences_by_length(sequences, length_bins=[100, 500, 1000, 5000, 10000, 20000]):
    """Group sequences into bins based on length"""
    grouped = defaultdict(list)
    
    for seq_id, seq in sequences:
        seq_len = len(seq)
        
        # Find appropriate bin
        bin_idx = 0
        for i, bin_size in enumerate(length_bins):
            if seq_len <= bin_size:
                bin_idx = i
                if i == 5:
                    print(seq_id, seq_len)
                break

        else:
            print("longer seq", seq_id, seq_len)
            bin_idx = len(length_bins)
        
        grouped[bin_idx].append((seq_id, seq))
    
    return grouped

# Group sequences
grouped_sequences = group_sequences_by_length(sequences)

print("Sequence distribution by length:")
for bin_idx, seqs in sorted(grouped_sequences.items()):
    print(f"Bin {bin_idx}: {len(seqs)} sequences")

sp|Q8WXI7|MUC16_HUMAN 14507
longer seq sp|Q8WZ42|TITIN_HUMAN 34350
sp|Q9I7U4|TITIN_DROME 18141
longer seq sp|A2ASS6|TITIN_MOUSE 35213
sp|G4SLH0|TTN1_CAEEL 18562
sp|M9MRD1|MS300_DROME 12345
sp|Q09165|DIG1_CAEEL 13100
sp|Q9H195|MUC3B_HUMAN 13477
Sequence distribution by length:
Bin 0: 3517 sequences
Bin 1: 47971 sequences
Bin 2: 22854 sequences
Bin 3: 7979 sequences
Bin 4: 75 sequences
Bin 5: 6 sequences
Bin 6: 2 sequences


In [ ]:
import shutil

def save_embeddings(embeddings_dict, output_dir, bin_idx=None, is_cumulative=False, is_final=False):
    """
    Save embeddings with proper naming, using temporary local storage
    """
    os.makedirs(output_dir, exist_ok=True)
    
    if is_final:
        filename = "embeddings_final"
    elif is_cumulative:
        filename = "embeddings_cumulative"
    elif bin_idx is not None:
        filename = f"embeddings_bin_{bin_idx}"
    else:
        filename = "embeddings"
    
    # Use temporary local directory
    temp_dir = "/tmp/embeddings_temp"
    os.makedirs(temp_dir, exist_ok=True)
    
    try:
        # Save to local temp first
        temp_pkl = os.path.join(temp_dir, f"{filename}.pkl")
        temp_npz = os.path.join(temp_dir, f"{filename}.npz")
        
        # Save as pickle
        with open(temp_pkl, 'wb') as f:
            pickle.dump(embeddings_dict, f)
        
        # Save as compressed numpy
        np.savez_compressed(temp_npz, **embeddings_dict)
        
        # Move to final destination
        final_pkl = os.path.join(output_dir, f"{filename}.pkl")
        final_npz = os.path.join(output_dir, f"{filename}.npz")
        
        shutil.copy2(temp_pkl, final_pkl)
        shutil.copy2(temp_npz, final_npz)
        
        # Clean up temp files
        os.remove(temp_pkl)
        os.remove(temp_npz)
        
        return final_pkl, final_npz
        
    except Exception as e:
        print(f"Error during save: {e}")
        # Keep temp files for debugging
        print(f"Temp files kept at: {temp_dir}")
        raise

In [ ]:
def load_progress(output_dir):
    """
    Load previous progress if exists
    Handles network drive issues by copying to local temp first
    """
    metadata_path = os.path.join(output_dir, "progress_metadata.pkl")
    cumulative_path = os.path.join(output_dir, "embeddings_cumulative.pkl")
    
    if os.path.exists(metadata_path) and os.path.exists(cumulative_path):
        try:
            # Try direct loading first
            with open(metadata_path, 'rb') as f:
                metadata = pickle.load(f)
            with open(cumulative_path, 'rb') as f:
                embeddings = pickle.load(f)
            return metadata, embeddings
        except (OSError, IOError) as e:
            print(f"Direct loading failed: {e}")
            print("Trying to copy to local temp first...")
            
            # Copy to local temp and load from there
            temp_dir = os.path.expanduser("~/tmp/embeddings_temp")
            os.makedirs(temp_dir, exist_ok=True)
            
            temp_metadata = os.path.join(temp_dir, "progress_metadata.pkl")
            temp_cumulative = os.path.join(temp_dir, "embeddings_cumulative.pkl")
            
            shutil.copy2(metadata_path, temp_metadata)
            shutil.copy2(cumulative_path, temp_cumulative)
            
            with open(temp_metadata, 'rb') as f:
                metadata = pickle.load(f)
            with open(temp_cumulative, 'rb') as f:
                embeddings = pickle.load(f)
            
            # Cleanup
            os.remove(temp_metadata)
            os.remove(temp_cumulative)
            
            return metadata, embeddings
    
    return None, {}

In [ ]:
# Define output directory
output_dir = "/Volumes/uc_rddw_dev_001/chi_test/extracted_embds/EMS2-embeddings/"

# Try to load previous progress
print("Checking for previous progress...")
metadata, all_embeddings = load_progress(output_dir)

if metadata:
    print(f"✓ Found previous progress!")
    print(f"  Last completed bin: {metadata['last_bin']}")
    print(f"  Sequences processed: {metadata['processed_sequences']}/{metadata['total_sequences']}")
    print(f"  Timestamp: {metadata['timestamp']}")
    start_bin = metadata['last_bin'] + 1
    processed_sequences = metadata['processed_sequences']
    print(f"\nResuming from Bin {start_bin}...")
else:
    print("No previous progress found. Starting fresh...")
    all_embeddings = {}
    start_bin = 0
    processed_sequences = 0

Checking for previous progress...
No previous progress found. Starting fresh...


In [ ]:
# Process each group with appropriate batch size
batch_sizes = {
    0: 64,   # Short sequences (<=100 aa)
    1: 16,   # Medium sequences (<=500 aa)
    2: 8,    # Long sequences (<=1000 aa)
    3: 4,    # Very long sequences (<=5000 aa)
    4: 2,    # Extremely long sequences (<=10000 aa)
    5: 1,    # Ultra long sequences (<=20000 aa)
    6: 1     # Longest sequences (>20000 aa)
}

total_sequences = sum(len(seqs) for seqs in grouped_sequences.values())

print(f"Total sequences to process: {total_sequences}")
print("="*80)

Total sequences to process: 82404


In [ ]:
for bin_idx, seqs in sorted(grouped_sequences.items()):
    # Skip already processed bins
    if bin_idx < start_bin:
        print(f"Skipping Bin {bin_idx} (already processed)")
        continue
    
    batch_size = batch_sizes.get(bin_idx, 1)
    total_batches = (len(seqs) + batch_size - 1) // batch_size
    
    print(f"\n{'='*80}")
    print(f"BIN {bin_idx}: Processing {len(seqs)} sequences")
    print(f"Batch size: {batch_size} | Total batches: {total_batches}")
    print(f"{'='*80}\n")
    
    start_time = time.time()
    
    try:
        # Extract embeddings for this bin
        embeddings = extract_embeddings_batch_with_progress(
            seqs, 
            model, 
            batch_converter, 
            device, 
            batch_size=batch_size
        )
        
        elapsed_time = time.time() - start_time
        processed_sequences += len(seqs)
        
        # Add to main dictionary
        all_embeddings.update(embeddings)
        
        print(f"\n✓ Bin {bin_idx} completed in {elapsed_time:.2f} seconds")
        print(f"  Average time per sequence: {elapsed_time/len(seqs):.3f} seconds")
        print(f"  Overall progress: {processed_sequences}/{total_sequences} "
              f"({processed_sequences/total_sequences*100:.1f}%)")
        
        # ========== SAVE AFTER EACH BIN ==========
        print(f"\n💾 Saving embeddings after Bin {bin_idx}...")
        
        # Save individual bin
        pkl_path, npz_path = save_embeddings(embeddings, output_dir, bin_idx=bin_idx)
        print(f"  ✓ Bin {bin_idx} saved:")
        print(f"    - {pkl_path}")
        print(f"    - {npz_path}")
        
        # Save cumulative embeddings
        pkl_path, npz_path = save_embeddings(all_embeddings, output_dir, is_cumulative=True)
        print(f"  ✓ Cumulative embeddings saved:")
        print(f"    - {pkl_path}")
        print(f"    - {npz_path}")
        
        # Save metadata
        metadata = {
            'total_sequences': total_sequences,
            'processed_sequences': processed_sequences,
            'bins_completed': bin_idx + 1,
            'last_bin': bin_idx,
            'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
            'total_embeddings': len(all_embeddings)
        }
        metadata_path = os.path.join(output_dir, "progress_metadata.pkl")
        with open(metadata_path, 'wb') as f:
            pickle.dump(metadata, f)
        print(f"  ✓ Progress metadata saved")
        print(f"  Total embeddings saved so far: {len(all_embeddings)}")
        # =========================================
        
        # Clean up memory
        del embeddings
        gc.collect()
        torch.cuda.empty_cache()
        
    except Exception as e:
        print(f"\n❌ ERROR in Bin {bin_idx}: {str(e)}")
        print(f"Progress saved up to Bin {bin_idx-1}")
        print(f"You can resume from Bin {bin_idx} by re-running this cell")
        raise

print(f"\n{'='*80}")
print(f"🎉 ALL BINS COMPLETED!")
print(f"Total embeddings extracted: {len(all_embeddings)}")
print(f"{'='*80}\n")


BIN 0: Processing 3517 sequences
Batch size: 64 | Total batches: 55



Processing sequences: 100%|████████████████████████████████████| 3517/3517 [15:25<00:00,  3.80seq/s]



✓ Bin 0 completed in 925.98 seconds
  Average time per sequence: 0.263 seconds
  Overall progress: 3517/82404 (4.3%)

💾 Saving embeddings after Bin 0...
  ✓ Bin 0 saved:
    - /Volumes/uc_rddw_dev_001/chi_test/extracted_embds/EMS2-embeddings/embeddings_bin_0.pkl
    - /Volumes/uc_rddw_dev_001/chi_test/extracted_embds/EMS2-embeddings/embeddings_bin_0.npz
  ✓ Cumulative embeddings saved:
    - /Volumes/uc_rddw_dev_001/chi_test/extracted_embds/EMS2-embeddings/embeddings_cumulative.pkl
    - /Volumes/uc_rddw_dev_001/chi_test/extracted_embds/EMS2-embeddings/embeddings_cumulative.npz
  ✓ Progress metadata saved
  Total embeddings saved so far: 3517

BIN 1: Processing 47971 sequences
Batch size: 16 | Total batches: 2999



Processing sequences:   8%|██▍                      

*** WARNING: max output size exceeded, skipping output. ***

Processing sequences:  37%|██████████▋                  | 17776/47971 [8:26:40<14:00:38,  1.67s/seq]

In [ ]:
# Final save
print("💾 Creating final output files...")
pkl_path, npz_path = save_embeddings(all_embeddings, output_dir, is_final=True)
print(f"✓ Final embeddings saved:")
print(f"  - {pkl_path}")
print(f"  - {npz_path}")

💾 Creating final output files...
✓ Final embeddings saved:
  - /Volumes/uc_rddw_dev_001/chi_test/extracted_embds/EMS2-embeddings/embeddings_final.pkl
  - /Volumes/uc_rddw_dev_001/chi_test/extracted_embds/EMS2-embeddings/embeddings_final.npz
